<a href="https://colab.research.google.com/github/TAKE-JP-17/Pytorch/blob/main/NYY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import ee
import geemap
import datetime
import calendar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
ee.Authenticate()
ee.Initialize(project='ee-f44081050')
map = geemap.Map()
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
shp = map.user_roi

In [5]:
def maskL5(image):
  # Bit 0 - Fill
  # Bit 1 - Dilated Cloud
  # Bit 2 - Cirrus
  # Bit 3 - Cloud
  # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

# Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B6').multiply(0.00341802).add(149.0)

# Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
        .addBands(thermalBands, None, True) \
        .updateMask(qaMask) \
        .updateMask(saturationMask)

def maskL8(image):
  # Bit 0 - Fill
  # Bit 1 - Dilated Cloud
  # Bit 2 - Cirrus
  # Bit 3 - Cloud
  # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

# Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

# Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
        .addBands(thermalBands, None, True) \
        .updateMask(qaMask) \
        .updateMask(saturationMask)

In [6]:
# Create a list Landsat 5, 4 monthly
coll1 = []

# Initial date
Date1 = datetime.date(1990, 1, 1)
# Final date
Date = datetime.date(1991, 12, 31)

# Months interval (To compute n monthly composite image, set K = n-1)
k = 3
while Date1 <= Date:
    outyr = Date1.year + (Date1.month - 1 + k) // 12
    outmonth = (Date1.month - 1 + k) % 12 + 1

    # Adjust the month and year
    if Date1.month + k > 12:
        outyr = Date1.year + 1
        outmonth = (Date1.month + k) % 12

    # Adjust the day to the last day of the month
    outday = calendar.monthrange(outyr, outmonth)[1]

    Date2 = datetime.date(outyr, outmonth, outday)

    iniDate = str(Date1)
    endDate = str(Date2)

    dataset = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
        .filterBounds(shp) \
        .filterDate(iniDate, endDate) \
        .map(maskL5)

    # select one image from collection to copy its properties to the composite image
    img1 = dataset.first()

    # Get Composite image date & season
    x = str(Date1.year)
    y = int(Date1.month)

    # Change according to months used to generate composite image
    Jan_apr = [1, 2, 3, 4]
    May_aug = [5, 6, 7, 8]
    sep_dec = [9, 10, 11, 12]

    if y in Jan_apr:
        s = str("Jan_apr")
    elif y in May_aug:
        s = str("May_aug")
    elif y in sep_dec:
        s = str("sep_dec")

    season = s + x

    size = dataset.size().getInfo()
    if size >= 1:
        composite = dataset.median().copyProperties(img1, ['system:time_start'])
        coll1.append(composite)

        print(season + "-" + str(size) + "__" + iniDate + "_" + endDate)
    else:
        print("No image in " + season)

    # Update Date1 for the next iteration
    Date1 = Date2 + datetime.timedelta(days=1)

collection_L5 = ee.ImageCollection.fromImages(coll1)

Jan_apr1990-6__1990-01-01_1990-04-30
May_aug1990-5__1990-05-01_1990-08-31
sep_dec1990-5__1990-09-01_1990-12-31
Jan_apr1991-6__1991-01-01_1991-04-30
May_aug1991-7__1991-05-01_1991-08-31
sep_dec1991-7__1991-09-01_1991-12-31


In [ ]:
#Compute MNDWI Images for both collection and merge them
#MNDWI = (Green - SWIR1) / (Green + SWIR1)
#NDVI = (NIR - Red) / (NIR + Red)
#NDBI = (SWIR1 - NIR) / (SWIR1 + NIR)
#PISI = 0.8192∗Blue − 0.5735∗NIR+0.075

def mndwi_fn_L5(image):
    mndwi = image.normalizedDifference(['SR_B2','SR_B5']).rename('mndwi')
    #mndwi = image.expression(
        #'0.8192 * blue - 0.5735 * nir + 0.075', {
            #'blue': image.select('SR_B1'),
            #'nir': image.select('SR_B4')
        #}).rename('mndwi')
    return image.addBands(mndwi)

def mndwi_fn_L8(image):
    mndwi = image.normalizedDifference(['SR_B3','SR_B6']).rename('mndwi')
    #mndwi = image.expression(
        #'0.8192 * blue - 0.5735 * nir + 0.075', {
            #'blue': image.select('SR_B2'),
            #'nir': image.select('SR_B5')
        #}).rename('mndwi')
    return image.addBands(mndwi)

mndwi_L5 = collection_L5.map(mndwi_fn_L5)
mndwi_L8 = collection_L8.map(mndwi_fn_L8)
mndwi_coll = mndwi_L5.merge(mndwi_L8)

In [ ]:
#Compute NDBI Images for both collection and merge them
#MNDWI = (Green - SWIR1) / (Green + SWIR1)
#NDVI = (NIR - Red) / (NIR + Red)
#NDBI = (SWIR1 - NIR) / (SWIR1 + NIR)
#PISI = 0.8192∗Blue − 0.5735∗NIR+0.075

def ndbi_fn_L5(image):
    ndbi = image.normalizedDifference(['SR_B5','SR_B4']).rename('ndbi')
    return image.addBands(ndbi)

def ndbi_fn_L8(image):
    ndbi = image.normalizedDifference(['SR_B6','SR_B5']).rename('ndbi')
    return image.addBands(ndbi)

ndbi_L5 = collection_L5.map(ndbi_fn_L5)
ndbi_L8 = collection_L8.map(ndbi_fn_L8)
ndbi_coll = ndbi_L5.merge(ndbi_L8)

In [ ]:
#Add a time band to the MNDWI collection
def Timeband(image):
    return image.addBands(image.metadata('system:time_start').divide(1000*60*60*24*365))

def clip_aoi_mndwi(image):
    return image.addBands(image.clip(shp).rename('mndwi_aoi'))

mndwi_collection = mndwi_coll.select('mndwi').map(clip_aoi_mndwi).map(Timeband)

In [ ]:
#Add a time band to the NDBI collection
def Timeband(image):
    return image.addBands(image.metadata('system:time_start').divide(1000*60*60*24*365))

def clip_aoi_ndbi(image):
    return image.addBands(image.clip(shp).rename('ndbi_aoi'))

ndbi_collection = ndbi_coll.select('ndbi').map(clip_aoi_ndbi).map(Timeband)

In [ ]:
# Compute Trend for the whole time series
trend_mndwi = mndwi_collection.select(['system:time_start','mndwi_aoi']).reduce(ee.Reducer.linearFit())
trend_ndbi = ndbi_collection.select(['system:time_start','ndbi_aoi']).reduce(ee.Reducer.linearFit())

#MNDWI
# 提取斜率
slope_mndwi = trend_mndwi.select('scale')

# 計算斜率的最小值和最大值
min_slope_mndwi = ee.Number(slope_mndwi.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=shp,
    scale=30,
    maxPixels=1e13
).get('scale'))

max_slope_mndwi = ee.Number(slope_mndwi.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=shp,
    scale=30,
    maxPixels=1e13
).get('scale'))

# 正規化斜率到-1到1之間
normalized_slope_mndwi = slope_mndwi.subtract(min_slope_mndwi).divide(max_slope_mndwi.subtract(min_slope_mndwi)).multiply(2).subtract(1).multiply(900)


# 合成正規化後的趨勢影像
normalized_trend_mndwi = trend_mndwi.addBands(normalized_slope_mndwi.rename('normalized_scale'), overwrite=True)

# 可視化正規化後的趨勢圖
map.addLayer(normalized_trend_mndwi, {'min': -900, 'max': 900, 'bands': ['normalized_scale'], 'palette': ['#d7191c','#fdae61','#ffffbf','#abd9e9','#2c7bb6']}, 'Normalized Trend_MNDWI')

In [ ]:
#NDBI
# 提取斜率
slope_ndbi = trend_ndbi.select('scale')

# 計算斜率的最小值和最大值
min_slope_ndbi = ee.Number(slope_ndbi.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=shp,
    scale=30,
    maxPixels=1e13
).get('scale'))

max_slope_ndbi = ee.Number(slope_ndbi.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=shp,
    scale=30,
    maxPixels=1e13
).get('scale'))

# 正規化斜率到-1到1之間
normalized_slope_ndbi = slope_ndbi.subtract(min_slope_ndbi).divide(max_slope_ndbi.subtract(min_slope_ndbi)).multiply(2).subtract(1).multiply(900)


# 合成正規化後的趨勢影像
normalized_trend_ndbi = trend_ndbi.addBands(normalized_slope_ndbi.rename('normalized_scale'), overwrite=True)

# 可視化正規化後的趨勢圖
map.addLayer(normalized_trend_ndbi, {'min': -900, 'max': 900, 'bands': ['normalized_scale'], 'palette': ['#66A000','#99B718','#ffffbf','#fdae61','#d7191c']}, 'Normalized Trend_NDBI')

In [ ]:
#正規化
# 合併 MNDWI, NDVI, NDBI 趨勢影像
combined_trend = normalized_trend_mndwi.select('normalized_scale').addBands(
    normalized_trend_ndbi.select('normalized_scale')
)
combined_trend = combined_trend.rename(['MNDWI_trend', 'NDBI_trend'])

# 計算每個波段的像元數量
pixel_count = combined_trend.reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=shp,
    scale=30,
    maxPixels=1e13
)

# 打印每個波段的像元數量
print('Pixel count for MNDWI:', pixel_count.get('MNDWI_trend').getInfo())
print('Pixel count for NDBI:', pixel_count.get('NDBI_trend').getInfo())

In [ ]:
pip show geemap

In [ ]:
import geemap
help(geemap.Map.add_legend)

In [ ]:
import IPython.display as display
import ee

# 生成圖片 URL
url = combined_trend.getThumbURL({
    'min': -900,
    'max': 900,
    'bands': ['MNDWI_trend'],
    'palette': ['#d7191c','#fdae61','#ffffbf','#abd9e9','#2c7bb6'],
    'region': shp,  # 使用 shp 幾何形狀
    'dimensions': 800,  # 調整縮略圖的尺寸
})

# 顯示圖片
display.Image(url=url)   # 顯示第一張圖片

In [ ]:
url2 = combined_trend.getThumbURL({
    'min': -900,
    'max': 900,
    'bands': ['NDBI_trend'],
    'palette': ['#d7191c','#fdae61','#ffffbf','#abd9e9','#2c7bb6'],
    'region': shp,  # 使用 shp 幾何形狀
    'dimensions': 800,  # 調整縮略圖的尺寸
})

display.Image(url=url2)

In [ ]:
from google.colab import files
import io
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colorbar import ColorbarBase
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

# 上傳 PNG 檔案到 Colab
uploaded = files.upload()

# 讀取上傳的檔案
file_name = list(uploaded.keys())[0]
image = Image.open(io.BytesIO(uploaded[file_name]))

# 生成顯示圖片的 subplot
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(image)

# 設置 color bar 的位置和尺寸
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

# 定義色碼映射
cmap = LinearSegmentedColormap.from_list(
    #MNDWI
    'custom', ['#d7191c', '#fdae61', '#ffffbf', '#abd9e9', '#2c7bb6'], N=256
    #NDVI
    #'custom', ['#d7191c' ,'#fdae61', '#ffffbf', '#99B718', '#66A000'], N=256
    #NDBI
    #'custom', ['#66A000' ,'#99B718', '#ffffbf', '#fdae61', '#d7191c'], N=256
)


# 設置 color bar 的顏色映射，去除數值標籤
cbar = ColorbarBase(cax, cmap=cmap, orientation='vertical', ticks=[])

# 在 color bar 的藍色右邊標示 Positive Trend（黑色字體）
cbar.ax.text(1.05, 1.0, 'Positive Trend, 900(m²/year)', color='black', fontsize=12, ha='left', va='center', rotation=0)

# 在 color bar 的紅色右邊標示 Negative Trend（黑色字體）
cbar.ax.text(1.05, 0.0, 'Negative Trend, -900(m²/year)', color='black', fontsize=12, ha='left', va='center', rotation=0)


# 隱藏 xy 軸
ax.set_axis_off()

# 顯示圖片
plt.show()

In [ ]:
# 上傳 PNG 檔案到 Colab
uploaded = files.upload()

# 讀取上傳的檔案
file_name = list(uploaded.keys())[0]
image = Image.open(io.BytesIO(uploaded[file_name]))

# 生成顯示圖片的 subplot
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(image)

# 設置 color bar 的位置和尺寸
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

# 定義色碼映射
cmap = LinearSegmentedColormap.from_list(
    #MNDWI
    'custom', ['#d7191c', '#fdae61', '#ffffbf', '#abd9e9', '#2c7bb6'], N=256
    #NDVI
    #'custom', ['#d7191c' ,'#fdae61', '#ffffbf', '#99B718', '#66A000'], N=256
    #NDBI
    #'custom', ['#66A000' ,'#99B718', '#ffffbf', '#fdae61', '#d7191c'], N=256
)


# 設置 color bar 的顏色映射，去除數值標籤
cbar = ColorbarBase(cax, cmap=cmap, orientation='vertical', ticks=[])

# 在 color bar 的藍色右邊標示 Positive Trend（黑色字體）
cbar.ax.text(1.05, 1.0, 'Positive Trend, 900(m²/year)', color='black', fontsize=12, ha='left', va='center', rotation=0)

# 在 color bar 的紅色右邊標示 Negative Trend（黑色字體）
cbar.ax.text(1.05, 0.0, 'Negative Trend, -900(m²/year)', color='black', fontsize=12, ha='left', va='center', rotation=0)


# 隱藏 xy 軸
ax.set_axis_off()

# 顯示圖片
plt.show()

In [ ]:
#輸出geotiff檔

# shp = shp.geometry()

task = ee.batch.Export.image.toDrive(image=normalized_trend_mndwi,
                    description='mndwi_trend',
                    scale=30,
                    region=map.user_roi,
                    fileNamePrefix='mndwi_trend',
                    crs='EPSG:4326',
                    fileFormat='GeoTIFF')
task.start()

In [ ]:
#輸出geotiff檔

# shp = shp.geometry()

task = ee.batch.Export.image.toDrive(image=normalized_trend_ndbi,
                    description='ndbi_trend',
                    scale=30,
                    region=map.user_roi,
                    fileNamePrefix='ndbi_trend',
                    crs='EPSG:4326',
                    fileFormat='GeoTIFF')
task.start()